Architecture of a FHIR app:
![image](https://cdn-images-1.medium.com/max/1200/1*qRd-H-cPHAGTB993sNLy3Q.png)
(from https://blog.heliossoftware.com/fhir-architectural-patterns-ae828b13d40c)

## BMI Calculator - ANSWERS

**Use Case**
A nurse practitioner wants to calculate a patient's body mass index (BMI). 


**Define Functional requirements**  
Step 1 - Define required clinical data elements  
- Patient height
- Patient weight
    
  
Step 2 - Identify FHIR Resources to support use case data (http://hl7.org/fhir/resourcelist.html)
- Patient height: Observation resource http://hl7.org/fhir/observation.html
- Patient weight: Observation resource http://hl7.org/fhir/observation.html

Step 3 - Identify data elements within each FHIR Resource definition necessary to support use case data

There are several with minor differences in meaning. Here we will use:
- Patient height: 8302-2
- Patient weight: 29463-7

## Initialize the environment to run the application

In [17]:
# Install the Python FHIR client package in the current Jupyter kernel using pip
import sys
!{sys.executable} -m pip install git+https://github.com/smart-on-fhir/client-py.git

  Cloning https://github.com/smart-on-fhir/client-py.git to /private/var/folders/rf/vnwqfm1s1bv29t28qfk6k9lm0000gn/T/pip-req-build-7lw8ijsg
  Running command git clone -q https://github.com/smart-on-fhir/client-py.git /private/var/folders/rf/vnwqfm1s1bv29t28qfk6k9lm0000gn/T/pip-req-build-7lw8ijsg
  Running command git submodule update --init --recursive -q
  Created wheel for fhirclient: filename=fhirclient-4.0.0-py2.py3-none-any.whl size=682982 sha256=8deca7a44ec4ca73a4f6e47118c2098c93b13db891ff4459c2816e7d27561a4b
  Stored in directory: /private/var/folders/rf/vnwqfm1s1bv29t28qfk6k9lm0000gn/T/pip-ephem-wheel-cache-vr_kjhvm/wheels/0a/b6/d6/645668a711b21c1934ecc4acb01f1fc9799afe8da3c9ee8f44
Successfully built fhirclient
  Attempting uninstall: fhirclient
    Found existing installation: fhirclient 3.2.0
    Uninstalling fhirclient-3.2.0:
      Successfully uninstalled fhirclient-3.2.0


In [1]:
# Import the client library that understands how to make FHIR calls and 
# interpret the results
#  Then print 'FHIR client library has been loaded'
from fhirclient import client
print('FHIR client library has been loaded')

FHIR client library has been loaded


## Find a patient with relevant variables

*Hint: Use the URL https://hapi.fhir.org/baseR4/[resource]?code=[loinc]*

Here we will use patient 431798

In [2]:
patientId = "431798"

## Load the data and get basic demographics

In [3]:
# Load the Patient from the database, and display demographics
#   Define the FHIR Endpoint. We define the name of this app (used in SMART 
#   calls), and the location of the server
settings = {
    'app_id': 'my_web_app',
    'api_base': 'https://hapi.fhir.org/baseR4'
}

In [4]:
# Create an instance of the FHIR client that points to this FHIR server. We use 
# this to make the calls.
db = client.FHIRClient(settings=settings)

In [5]:
# Perform a GET (read) on the patient with the ID you identified earlier
from fhirclient.models.patient import Patient
patient = Patient.read(patientId, db.server)

In [42]:
patient.as_json()

{'id': '431798',
 'meta': {'lastUpdated': '2019-12-16T15:28:18.377+00:00',
  'source': '#1D6a5ZvM9ldncHf3',
  'versionId': '1'},
 'extension': [{'extension': [{'url': 'ombCategory',
     'valueCoding': {'code': '2106-3',
      'display': 'White',
      'system': 'urn:oid:2.16.840.1.113883.6.238'}},
    {'url': 'text', 'valueString': 'White'}],
   'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'},
  {'extension': [{'url': 'ombCategory',
     'valueCoding': {'code': '2186-5',
      'display': 'Not Hispanic or Latino',
      'system': 'urn:oid:2.16.840.1.113883.6.238'}},
    {'url': 'text', 'valueString': 'Not Hispanic or Latino'}],
   'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity'},
  {'url': 'http://hl7.org/fhir/StructureDefinition/patient-mothersMaidenName',
   'valueString': 'Drusilla108 McCullough561'},
  {'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex',
   'valueCode': 'M'},
  {'url': 'http://hl7.org/fhir/Str

## *Optional: get basic demographics*

In [7]:
# Patient name. 
# Show the name object so we can see what's in it
print(patient.name)

The patient's name is a list of `HumanName`s. We can find out about the structure of this data type on the FHIR website: https://www.hl7.org/fhir/datatypes.html#HumanName

In [8]:
# Show the contents of each object in the list
for name in patient.name:
    print(name.as_json())

{'family': 'Cronin387', 'given': ['Olin642'], 'prefix': ['Mr.'], 'use': 'official'}


In [9]:
# We see that a Patient can have more than one name (i.e. Patient.name is a list)
# and each name can have more than one given name (i.e. given is also a list).
# Let's use the first name in the list and the first given name:
first_name = patient.name[0].given[0]
last_name = patient.name[0].family
print(first_name, last_name)

Olin642 Cronin387


In [10]:
# Patient's gender
gender = patient.gender
gender

'male'

In [12]:
# Define variable dob as the patient's date of birth
dob = patient.birthDate.date
print(dob)

1952-01-10


In [13]:
# We'd like to know the patient's age. The EHR contains the date of birth only, 
# so we have to calculate the age using the dob and today's date.
# For this we need the datetime library
import datetime

# Define and print today variable.
today = datetime.date.today()
print(today)

2020-09-09


In [14]:
# Calculate the patient's age using the relativedelta method of the dateutil 
# module
from dateutil.relativedelta import relativedelta

delta = relativedelta(today, dob)
age = delta.years
age

68

In [15]:
# Print patient name, gender, DOB, age, and today's date
print("Patient's Name(s) =", first_name, last_name)
print("Gender =", gender)
print("DOB =", dob)
print("Today's Date =", today)
print("Patient's age =", age)

Patient's Name(s) = Olin642 Cronin387
Gender = male
DOB = 1952-01-10
Today's Date = 2020-09-09
Patient's age = 68


## Query database for relevant information

In [18]:
#Query database for weight (using the LOINC code for weight) and print it with units

# allows search of Observation resources
from fhirclient.models.observation import Observation

# specify the search, setting the patient, the code we want and the sort order 
# (descending order by date)
search = Observation.where(
    struct={'patient': "Patient/"+patientId, 'code': "29463-7", '_sort': '-date'})

# show the actual url that is generated
url = search.construct()
print('Query: ', url)

Query:  Observation?patient=Patient%2F431798&code=29463-7&_sort=-date


Note that all we need to do to execute this query is to open the URL. Try it by visiting the following link in your browser:

In [25]:
print(db.server.base_uri + url)

https://hapi.fhir.org/baseR4/Observation?patient=Patient%2F431798&code=29463-7&_sort=-date


In [27]:
# Run the query
weight_observations = search.perform_resources(db.server)
if not weight_observations:
    print("No resources returned")

# because we sorted by -date, the most recent weight observation comes first
latest_weight = weight_observations[0]
latest_weight.as_json()

{'id': '432348',
 'meta': {'lastUpdated': '2019-12-16T15:28:18.377+00:00',
  'source': '#1D6a5ZvM9ldncHf3',
  'versionId': '1'},
 'category': [{'coding': [{'code': 'vital-signs',
     'display': 'vital-signs',
     'system': 'http://terminology.hl7.org/CodeSystem/observation-category'}]}],
 'code': {'coding': [{'code': '29463-7',
    'display': 'Body Weight',
    'system': 'http://loinc.org'}],
  'text': 'Body Weight'},
 'effectiveDateTime': '2019-12-12T18:20:26+01:00',
 'encounter': {'reference': 'Encounter/432345'},
 'issued': '2019-12-12T18:20:26.977+01:00',
 'status': 'final',
 'subject': {'reference': 'Patient/431798'},
 'valueQuantity': {'code': 'kg',
  'system': 'http://unitsofmeasure.org',
  'unit': 'kg',
  'value': 84},
 'resourceType': 'Observation'}

We can also look at the definition of the `Observation` class here: https://www.hl7.org/fhir/observation.html#resource.
There we can see that the actual value will be in the `valueQuantity` field, which is of type `Quantity` (https://www.hl7.org/fhir/datatypes.html#quantity), which has fields `value` and `unit` 

In [28]:
weight_value = latest_weight.valueQuantity.value
weight_unit = latest_weight.valueQuantity.unit
print(weight_value, weight_unit)

84 kg


In [29]:
# Query the database for height and print it with units
#  Remember to find the LOINC for height

# specify the new search, setting the patient, the code we want and the sort
# order (descending order by date)
search = Observation.where(
    struct={'patient': "Patient/"+patientId, 'code': "8302-2", '_sort': '-date'})

# show the actual url that is generated
url = search.construct()
print('Query: ', url)

Query:  Observation?patient=Patient%2F431798&code=8302-2&_sort=-date


In [30]:
# height observations have a structure that is the same as the weight observation
# structure
height_observations = search.perform_resources(db.server)
if not height_observations:
    print("No resources returned")
    
latest_height = height_observations[0]
latest_height.as_json()

{'id': '432346',
 'meta': {'lastUpdated': '2019-12-16T15:28:18.377+00:00',
  'source': '#1D6a5ZvM9ldncHf3',
  'versionId': '1'},
 'category': [{'coding': [{'code': 'vital-signs',
     'display': 'vital-signs',
     'system': 'http://terminology.hl7.org/CodeSystem/observation-category'}]}],
 'code': {'coding': [{'code': '8302-2',
    'display': 'Body Height',
    'system': 'http://loinc.org'}],
  'text': 'Body Height'},
 'effectiveDateTime': '2019-12-12T18:20:26+01:00',
 'encounter': {'reference': 'Encounter/432345'},
 'issued': '2019-12-12T18:20:26.977+01:00',
 'status': 'final',
 'subject': {'reference': 'Patient/431798'},
 'valueQuantity': {'code': 'cm',
  'system': 'http://unitsofmeasure.org',
  'unit': 'cm',
  'value': 175.8},
 'resourceType': 'Observation'}

In [31]:
height_value = latest_height.valueQuantity.value
height_unit = latest_height.valueQuantity.unit
print(height_value, height_unit)

175.8 cm


## Calculate BMI

In [32]:
# We now have all the information we need to calculate BMI.
# Calculate the BMI to one decimal
bmi = weight_value/((height_value/100)**2)
bmi = round(bmi, 1)
bmi

27.2

In [33]:
# check BMI category
#    Hint: See https://www.nhlbi.nih.gov/health/educational/lose_wt/risk.htm
#    for categories
if bmi < 18.5:
    print("BMI indicates patient is underweight")
elif bmi <= 24.9:
    print("BMI indicates patient is within normal weight range")
elif bmi <= 29.9:
    print("BMI indicates patient is overweight")
else:
    print("BMI indicates patient is obese")

BMI indicates patient is overweight
